# 前処理

購買データ、推薦データを以下の形式に変換する

table: transaction
| user_id | t | store_id | item_id |
|---------|---|----------|---------|
| 2375    | 0 | 0        | 0       |
| 2375    | 0 | 1        | 1       |
| 2375    | 0 | 1        | 1       |
| 2375    | 1 | 0        | 1       |
| 2380    | 0 | 1        | 0       |

table: recommendation
| t | store_id | item_ids     |
|---|----------|--------------|
| 0 | 0        | [0, 1, 3, 5] |
| 1 | 0        | [0, 3, 4, 5] |
| 3 | 1        | [2, 4, 5,6]  |

In [1]:
import pandas as pd

In [2]:
def data_path(file: str) -> str:
    return f"../data/dunnhumby_The-Complete-Journey/dunnhumby_The-Complete-Journey CSV/{file}"

transaction_df = pd.read_csv(data_path("transaction_data.csv"))
causal_df = pd.read_csv(data_path("causal_data.csv"))

Filtering
- shops that have at least one visitor for each week
- items recommended for at least one week on average among the shops
- items that existed for at least half the period (47 weeks)
- users visiting more than one store in at least five weeks

In [3]:
# 推薦があった週だけ残す
valid_weeks = set(causal_df.WEEK_NO.unique())
transaction_df = transaction_df[transaction_df.WEEK_NO.isin(valid_weeks)]
causal_df = causal_df[causal_df.WEEK_NO.isin(valid_weeks)]

In [4]:
causal_df = causal_df[causal_df.display != "0"]

In [5]:
# shops that have at least one visitor for each week
a = transaction_df.groupby("STORE_ID")["WEEK_NO"].nunique()
valid_stores = set(a[a == transaction_df["WEEK_NO"].nunique()].index)
transaction_df = transaction_df[transaction_df["STORE_ID"].isin(valid_stores)]
causal_df = causal_df[causal_df["STORE_ID"].isin(valid_stores)]

# items recommended for at least one week on average among the shops
a = causal_df["PRODUCT_ID"].value_counts()
valid_items1 = set(a[a >= len(valid_stores)].index)

# items that existed for at least half the period (47 weeks)
a = transaction_df.groupby("PRODUCT_ID")["WEEK_NO"].nunique()
valid_items2 = set(a[a > transaction_df["WEEK_NO"].nunique() / 2].index)

# users visiting more than one store in at least five weeks
a = transaction_df.groupby(["household_key", "WEEK_NO"])["STORE_ID"].nunique()
a = a[a > 1].groupby("household_key").count()
valid_users = set(a[a >= 5].index)

valid_items = valid_items1 & valid_items2
print(len(valid_users), len(valid_items), len(valid_stores))

transaction_df = transaction_df[
    transaction_df.PRODUCT_ID.isin(valid_items)
    & transaction_df.household_key.isin(valid_users)
].reset_index(drop=True)
causal_df = causal_df[causal_df.PRODUCT_ID.isin(valid_items)].reset_index(drop=True)

1096 3664 100


In [6]:
transaction_df.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,1193,27636995049,55,942088,3,1.0,334,-0.64,518,9,0.0,0.0
1,1193,27636995049,55,985893,1,2.0,334,-0.99,518,9,0.0,0.0
2,1193,27636995049,55,1013928,2,20.0,334,0.00,518,9,0.0,0.0
3,1193,27636995049,55,1115874,3,1.0,334,-0.63,518,9,0.0,0.0
4,1193,27636995049,55,5569230,1,3.5,334,-1.09,518,9,0.0,0.0


In [7]:
causal_df.head()

,PRODUCT_ID,STORE_ID,WEEK_NO,display,mailer
0,818981,288,25,5,D
1,818981,288,26,5,0
2,818981,288,27,5,0
3,818981,288,43,5,D
4,818981,288,47,5,A


In [8]:
transaction_df = transaction_df[["household_key", "WEEK_NO", "STORE_ID", "PRODUCT_ID"]]
causal_df = causal_df.groupby(["WEEK_NO", "STORE_ID"])["PRODUCT_ID"].unique().rename("PRODUCT_IDS").to_frame().reset_index()

In [9]:
causal_df["PRODUCT_IDS"] = causal_df["PRODUCT_IDS"].apply(lambda s: " ".join(map(str, s)))

In [10]:
transaction_df = transaction_df.rename(
    columns={
        "household_key": "user_id",
        "WEEK_NO": "t",
        "STORE_ID": "store_id",
        "PRODUCT_ID": "item_id",
    }
)
causal_df = causal_df.rename(
    columns={"WEEK_NO": "t", "STORE_ID": "store_id", "PRODUCT_IDS": "item_ids"}
)

In [11]:
transaction_df.to_csv("../data/preprocessed/transaction.csv", index=False)
causal_df.to_csv("../data/preprocessed/recommendation.csv", index=False)